In [1]:
import numpy as np 
import os
import pandas as pd 
import time
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn import tree
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn import metrics
from sklearn.feature_selection import chi2
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import time

In [2]:
import os
for dirname, _, filenames in os.walk('/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv
/home/pranav/Desktop/ML/Project/archive/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
low_memory=False
df1 = pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv")
df2=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv")
df3=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Friday-WorkingHours-Morning.pcap_ISCX.csv")
df4=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Monday-WorkingHours.pcap_ISCX.csv")
df5=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv")
df6=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv")
df7=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Tuesday-WorkingHours.pcap_ISCX.csv")
df8=pd.read_csv("/home/pranav/Desktop/ML/Project/MachineLearningCSV/MachineLearningCVE/Wednesday-workingHours.pcap_ISCX.csv")

In [4]:
df=pd.concat([df1,df2,df3,df4,df5,df6,df7,df8])

In [5]:
df[' Label'].value_counts()

 Label
BENIGN                        2273097
DoS Hulk                       231073
PortScan                       158930
DDoS                           128027
DoS GoldenEye                   10293
FTP-Patator                      7938
SSH-Patator                      5897
DoS slowloris                    5796
DoS Slowhttptest                 5499
Bot                              1966
Web Attack � Brute Force         1507
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [6]:
df.shape

(2830743, 79)

In [7]:
# Remove duplicate rows
df = df.drop_duplicates(keep="first")

# Remove rows with NaN, positive infinity, or negative infinity
df = df[~df.isin([np.nan, np.inf, -np.inf]).any(axis=1)]

In [8]:
df.shape

(2520798, 79)

In [9]:
# Convert int64 into int32 and float64 into float32
integer = []
f = []
for i in df.columns[:-1]:
    if df[i].dtype == "int64":
        integer.append(i)
    elif df[i].dtype == "float64":
        f.append(i)

df[integer] = df[integer].astype("int32")
df[f] = df[f].astype("float32")

In [10]:
# Exclude non-numeric columns
numeric_df = df.select_dtypes(include=['int32', 'float32'])

# Calculate the correlation matrix for numeric columns
corr_matrix = numeric_df.corr()

# Define the correlation threshold
threshold = 0.85

# Find highly correlated features
corr_features = set()
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > threshold:
            colname = corr_matrix.columns[i]
            corr_features.add(colname)

# Drop highly correlated features from the original DataFrame
df.drop(corr_features, axis=1, inplace=True)


In [11]:
df.shape

(2520798, 44)

In [12]:
df[' Label'].value_counts()

 Label
BENIGN                        2095057
DoS Hulk                       172846
DDoS                           128014
PortScan                        90694
DoS GoldenEye                   10286
FTP-Patator                      5931
DoS slowloris                    5385
DoS Slowhttptest                 5228
SSH-Patator                      3219
Bot                              1948
Web Attack � Brute Force         1470
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: count, dtype: int64

In [13]:
#Create a new dataframe with the attacks that have more than 10000 instances in the original dataframe
df1=df.groupby(' Label').filter(lambda x:len(x)>10000)
df1[' Label'].value_counts()


 Label
BENIGN           2095057
DoS Hulk          172846
DDoS              128014
PortScan           90694
DoS GoldenEye      10286
Name: count, dtype: int64

In [14]:
#Create a new dataframe with the attacks that have less than 10000 instances in the original dataframe
df2=df.groupby(' Label').filter(lambda x:len(x)<10000)
df2[' Label'].value_counts()

 Label
FTP-Patator                   5931
DoS slowloris                 5385
DoS Slowhttptest              5228
SSH-Patator                   3219
Bot                           1948
Web Attack � Brute Force      1470
Web Attack � XSS               652
Infiltration                    36
Web Attack � Sql Injection      21
Heartbleed                      11
Name: count, dtype: int64

In [15]:
x = df1.drop([' Label'],axis=1)
y = df1[' Label']

In [16]:
unique, countsa = np.unique(y, return_counts=True)
print(list(zip(unique,countsa)))

[('BENIGN', 2095057), ('DDoS', 128014), ('DoS GoldenEye', 10286), ('DoS Hulk', 172846), ('PortScan', 90694)]


In [17]:
#Down sample the first dataframe (attacks with more than 10000) so that each calss has around 10000 instances. A random undersampling is oerformed for the purpose.
rus = RandomUnderSampler(random_state=0)
rus.fit(x, y)
Xn, yn = rus.fit_resample(x, y)

In [18]:
unique, countsb = np.unique(yn, return_counts=True)
print(list(zip(unique,countsb)))

[('BENIGN', 10286), ('DDoS', 10286), ('DoS GoldenEye', 10286), ('DoS Hulk', 10286), ('PortScan', 10286)]


In [19]:
df1=pd.concat([Xn,yn],axis=1)
df1[' Label'].value_counts()

 Label
BENIGN           10286
DDoS             10286
DoS GoldenEye    10286
DoS Hulk         10286
PortScan         10286
Name: count, dtype: int64

In [20]:
df=pd.concat([df1,df2])
df[' Label'].value_counts()

 Label
BENIGN                        10286
DDoS                          10286
DoS GoldenEye                 10286
DoS Hulk                      10286
PortScan                      10286
FTP-Patator                    5931
DoS slowloris                  5385
DoS Slowhttptest               5228
SSH-Patator                    3219
Bot                            1948
Web Attack � Brute Force       1470
Web Attack � XSS                652
Infiltration                     36
Web Attack � Sql Injection       21
Heartbleed                       11
Name: count, dtype: int64

In [21]:
x = df.drop([' Label'],axis=1)
y = df[' Label']

In [22]:
#Change the categorical labels into numerical
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
le = LabelEncoder()
y=le.fit_transform(df[' Label'].values)

In [23]:
unique, counts1 = np.unique(y, return_counts=True)
print(list(zip(unique,counts1)))

[(0, 10286), (1, 1948), (2, 10286), (3, 10286), (4, 10286), (5, 5228), (6, 5385), (7, 5931), (8, 11), (9, 36), (10, 10286), (11, 3219), (12, 1470), (13, 21), (14, 652)]


In [24]:
#Oversampling with SMOTE to balance the dataset
from imblearn.over_sampling import SMOTE
oversample = SMOTE( random_state=42)
Xn,Yn = oversample.fit_resample(x, y)

In [25]:
unique, counts = np.unique(Yn, return_counts=True)
print(list(zip(unique,counts)))

[(0, 10286), (1, 10286), (2, 10286), (3, 10286), (4, 10286), (5, 10286), (6, 10286), (7, 10286), (8, 10286), (9, 10286), (10, 10286), (11, 10286), (12, 10286), (13, 10286), (14, 10286)]


In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(Xn,Yn,test_size=0.30,random_state=0)

In [27]:
# Create an instance of SimpleImputer with 'mean' strategy to replace NaN values
imputer = SimpleImputer(strategy='mean')

# Fit the imputer to X_train and transform X_train and X_test with it
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [30]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score, recall_score, precision_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# Assuming you have your data in X and Y, and you split it into training and testing sets
# X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create the AdaBoost classifier with DecisionTree base estimator
base_estimator = DecisionTreeClassifier(max_depth=20)
adaboost = AdaBoostClassifier(estimator=base_estimator, random_state=42)

# Train the AdaBoost classifier
adaboost.fit(X_train, Y_train)

# Use the trained model for predictions
adapreds = adaboost.predict(X_test)

# Cross-validation accuracy score
cross_val_acc_scores = cross_val_score(adaboost, X_train, Y_train, cv=3, scoring='accuracy')
print("Cross Validation Accuracy Score:", cross_val_acc_scores)

# Accuracy score
accuracy = accuracy_score(Y_test, adapreds)
print("Accuracy Score:", accuracy)

# F1 score (weighted for multiclass)
f1_weighted = f1_score(Y_test, adapreds, average='weighted')
print("F1 Score (Weighted):", f1_weighted)

# Recall score (weighted for multiclass)
recall_weighted = recall_score(Y_test, adapreds, average='weighted')
print("Recall Score (Weighted):", recall_weighted)

# Precision score (weighted for multiclass)
precision_weighted = precision_score(Y_test, adapreds, average='weighted')
print("Precision Score (Weighted):", precision_weighted)

# Classification report
print("Classification Report:")
print(classification_report(Y_test, adapreds))

# F1 Macro Score
f1_macro = f1_score(Y_test, adapreds, average='macro')
print("F1 Macro Score:", f1_macro)

# F1 Micro Score
f1_micro = f1_score(Y_test, adapreds, average='micro')
print("F1 Micro Score:", f1_micro)


Cross Validation Accuracy Score: [0.97847282 0.97955612 0.97930613]
Accuracy Score: 0.9816363125715644
F1 Score (Weighted): 0.9816462225132941
Recall Score (Weighted): 0.9816363125715644
Precision Score (Weighted): 0.9816619421067284
Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      3075
           1       1.00      1.00      1.00      3114
           2       1.00      1.00      1.00      3044
           3       1.00      1.00      1.00      3025
           4       1.00      1.00      1.00      3111
           5       1.00      0.99      1.00      3076
           6       1.00      1.00      1.00      3080
           7       1.00      1.00      1.00      3043
           8       1.00      1.00      1.00      3092
           9       1.00      1.00      1.00      3135
          10       1.00      1.00      1.00      3081
          11       1.00      1.00      1.00      3151
          12       0.88      0.87      0